In [1]:
import pandas as pd
import numpy as np

In [2]:
filepath = "malware_detection_dataset.csv"

In [3]:
df = pd.read_csv(filepath)

### Split the Dataset into X and y

In [4]:
X = df.drop('label',axis=1).head(400000)
y = df['label'].head(400000)

In [5]:
X

,duration,orig_bytes,resp_bytes,missed_bytes,orig_pkts,orig_ip_bytes,resp_pkts,resp_ip_bytes,proto_icmp,proto_tcp,...,conn_state_RSTOS0,conn_state_RSTR,conn_state_RSTRH,conn_state_S0,conn_state_S1,conn_state_S2,conn_state_S3,conn_state_SF,conn_state_SH,conn_state_SHR
0,0.114184,48.000000,48.0,0,1,76,1,76,0,0,...,0,0,0,0,0,0,0,1,0,0
1,160.367579,7536.000000,0.0,0,24,8208,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,0.016986,48.000000,48.0,0,1,76,1,76,0,0,...,0,0,0,0,0,0,0,1,0,0
3,0.003497,0.000000,0.0,0,5,212,3,144,0,1,...,0,0,0,0,0,0,0,1,0,0
4,0.036724,34.000000,311.0,0,1,62,1,339,0,0,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
399995,14.659356,156.959688,0.0,0,1,40,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
399996,14.619064,156.624187,0.0,0,1,40,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
399997,14.578772,156.288687,0.0,0,1,40,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0
399998,14.538481,155.953186,0.0,0,1,40,0,0,0,1,...,0,0,0,1,0,0,0,0,0,0


In [6]:
y

0                            benign
1                            benign
2                            benign
3                            benign
4                            benign
                    ...            
399995    PartOfAHorizontalPortScan
399996    PartOfAHorizontalPortScan
399997    PartOfAHorizontalPortScan
399998    PartOfAHorizontalPortScan
399999    PartOfAHorizontalPortScan
Name: label, Length: 400000, dtype: object

### Splitting the X and y Dataset into the Training set and Test set

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 1)

In [8]:
y_test

164217    PartOfAHorizontalPortScan
136326    PartOfAHorizontalPortScan
139035    PartOfAHorizontalPortScan
184089    PartOfAHorizontalPortScan
49577     PartOfAHorizontalPortScan
                    ...            
356732    PartOfAHorizontalPortScan
298300                        Okiru
342305    PartOfAHorizontalPortScan
246290                        Okiru
259816                        Okiru
Name: label, Length: 80000, dtype: object

In [9]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

(320000, 24)
(80000, 24)
(320000,)
(80000,)


### Feature Scaling

In [11]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [12]:
import timeit
start = timeit.default_timer()

In [13]:
#Fit SVM to the Training set
from sklearn.svm import SVC
svm_clf = SVC(random_state = 0, C=1.0, cache_size=1500, verbose = True) # Create SVM classifier object
svm_clf.fit(X_train, y_train) # Train SVM Classifier

[LibSVM]

SVC(cache_size=1500, random_state=0, verbose=True)

In [14]:
end = timeit.default_timer()
print("total runtime of the program is: ",end-start,'seconds')

total runtime of the program is:  15302.785966899999 seconds


In [15]:
### Predicting the test results
svm_pred = svm_clf.predict(X_test)   #y_predict

In [16]:
svm_pred

array(['PartOfAHorizontalPortScan', 'PartOfAHorizontalPortScan',
       'PartOfAHorizontalPortScan', ..., 'PartOfAHorizontalPortScan',
       'PartOfAHorizontalPortScan', 'PartOfAHorizontalPortScan'],
      dtype=object)

In [17]:
print()
end = timeit.default_timer()
print("total runtime of the program is: ",end-start,'seconds')


total runtime of the program is:  19697.4052581 seconds


In [18]:
##Confusion matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, svm_pred)
print(cm)

[[  777     0     0     0     0     0]
 [    0     0     0     0  1687     3]
 [    0     0     0     0     7    14]
 [    0     0     0     0 19890    25]
 [  218     0     0     0 46590   110]
 [   59     0     0     0   859  9761]]


In [19]:
### Final Results
from sklearn.metrics import classification_report
print(classification_report(y_test,svm_pred,zero_division=0))

                           precision    recall  f1-score   support

                   Attack       0.74      1.00      0.85       777
                      C&C       0.00      0.00      0.00      1690
            C&C-HeartBeat       0.00      0.00      0.00        21
                    Okiru       0.00      0.00      0.00     19915
PartOfAHorizontalPortScan       0.67      0.99      0.80     46918
                   benign       0.98      0.91      0.95     10679

                 accuracy                           0.71     80000
                macro avg       0.40      0.48      0.43     80000
             weighted avg       0.53      0.71      0.61     80000



In [20]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, svm_pred)


0.7141